In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time
import graphviz
import pydot

In [ ]:
# 파일 읽어오기
df_activity_list = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/NetworkX/DSME_ACTIVITY_LIST.csv", encoding='utf-8')
df_relation_list = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/NetworkX/DSME_RELATION.csv", encoding='utf-8')

# ACTIVITYCODE를 인덱스로 설정
df_activity_list.set_index('ACTIVITYCODE', inplace = True)

# activity를 node로 가지는 그래프 생성
act_network = nx.DiGraph()
for idx, row in df_activity_list.iterrows():
    act_network.add_node(idx)
    for item, col in zip(row, df_activity_list.columns):
        act_network.nodes[idx][col] = item

# relation으로 edge 연결 (현재 node에 없는 activity가 있는 경우, node를 자동으로 생성함)
for idx, row in df_relation_list.iterrows():
    act_network.add_edge(row['PREVACTIVITYCODE'], row['NEXTACTIVITYCODE'])
    for item, col in zip(row, df_relation_list.columns):
        if col != 'PREVACTIVITYCODE' and col != 'NEXTACTIVITYCODE':
            act_network.edges[row['PREVACTIVITYCODE'], row['NEXTACTIVITYCODE']][col] = item

In [ ]:
# 시각화를 위한 기본 정보 입력
act_network.graph['graph']={'rankdir':'LR'}
act_network.graph['node']={'shape':'circle'}
act_network.graph['edges']={'arrowsize':'4.0'}

In [ ]:
# 탑재 activity들 리스트에 따로 저장
list_temp_nodes = []
for node in act_network.nodes():
    try:
        if '탑재' == act_network.nodes[node]['DESCRIPTION'][-2:]: 
            list_temp_nodes.append(node)
            # print(act_network.nodes[node])
    except:
        pass

list_temp_nodes.append('K/L')
list_temp_nodes.append('L/C')

erect_network = act_network.subgraph(list_temp_nodes)


In [ ]:
# CPM 테스트를 위해, 가장 많은 노드가 연결된 subgraph만 main_network로 따로 저장
for subgraph in nx.weakly_connected_components(act_network):
    if len(subgraph) == 568:
        main_network = act_network.subgraph(subgraph)

# 한 그래프의 subgraph를 저장한 경우, node 및 edge의 추가/제거가 안되기 때문에, 추가/제거가 가능하도록 새로운 그래프로 정의
main_network = nx.DiGraph(main_network)

In [ ]:
# main_network 중 L/C를 제외하고 후행 공정이 정의되지 않은 node 삭제
all_prunned = False
while all_prunned == False:
    remove_node_list = []
    for node in main_network.nodes:
        if node != 'L/C':
            if main_network.out_degree(node) == 0:
                remove_node_list.append(node)

    main_network.remove_nodes_from(remove_node_list)
    if len(remove_node_list) == 0:
        all_prunned = True

In [ ]:
# 전처리 된 main_network 시각화 확인
src = graphviz.Source(nx.nx_pydot.to_pydot(main_network))
src.render("/content/gdrive/My Drive/Colab Notebooks/NetworkX/main_network_prunned2.gv", view=True)

'/content/gdrive/My Drive/Colab Notebooks/NetworkX/main_network_prunned2.gv.pdf'

In [ ]:
# 탑재 공정 파란색으로 표시 후 시각화
for node in erect_network.nodes:
    try:
        main_network.nodes[node]['color'] = 'blue'
    except:
        pass

src = graphviz.Source(nx.nx_pydot.to_pydot(main_network))
src.render("/content/gdrive/My Drive/Colab Notebooks/NetworkX/main_network_prunned_color.gv", view=True)

In [ ]:
# CPM 과정에서 사용하기 위해 각 노드에 'duration' 특성 추가
for node in main_network.nodes:
    try:
        main_network.nodes[node]['duration'] = main_network.nodes[node]['PLANDURATION'] # 변수 이름의 통일을 위함
    except:
        main_network.nodes[node]['duration'] = 1 # 작업 소요 시간이 정해져 있지 않은 경우 기본값으로 1을 설정

In [ ]:
# Critical Path 계산을 위한 함수

# 마지막 노드에서 backward로 진행하면서 LF, LS 계산하는 함수
def backward(DG):
    node_list = []
    for node in nx.topological_sort(DG):
        node_list.append(node)
    node_list.reverse()

    for node in node_list:
        lf = min([DG.nodes[j]['LS'] for j in DG.successors(node)], default = 0)
        DG.nodes[node]['LF'] = lf
        DG.nodes[node]['LS'] = lf - DG.nodes[node]['duration']

# 처음 backward로 진행이 끝난 후 찾은 가장 작업시간이 긴 경로의 길이를 반환하는 함수
def compute_temp_length(DG):
    return -min(nx.get_node_attributes(DG, 'LS').values())

# 첫번째 노드에서 forward로 진행하면서 ES, EF 계산하는 함수
def forward(DG, default_length):
    node_list = []
    for node in nx.topological_sort(DG):
        node_list.append(node)

    for node in node_list:
        es = max([DG.nodes[j]['EF'] for j in DG.predecessors(node)], default = -default_length)
        DG.nodes[node]['ES'] = es
        DG.nodes[node]['EF'] = es + DG.nodes[node]['duration']

# 여유 시간 계산하는 함수
def compute_slack(DG):
    for node in DG:
        DG.nodes[node]['slack'] = DG.nodes[node]['LF'] - DG.nodes[node]['EF']

# critical path의 node 집합을 반환하는 함수
def compute_critical_path(DG):
    graph = set()
    for node in DG:
        if DG.nodes[node]['slack'] == 0:
            graph.add(node)
    return graph



In [ ]:
# Critical Path 계산 수행
backward(main_network)
critical_path_length = compute_temp_length(main_network)
forward(main_network, default_length = critical_path_length)
compute_slack(main_network)
critical_path = compute_critical_path(main_network)

In [ ]:
# Critical Path 상의 노드들에 대해, ES, EF, LS, LF, duration, Slack을 각각 출력
for node in critical_path:
    print(main_network.nodes[node]['ES'], main_network.nodes[node]['EF'], main_network.nodes[node]['LS'], main_network.nodes[node]['LF'], main_network.nodes[node]['duration'], main_network.nodes[node]['slack'])

-8 -7 -8 -7 1 0
-13 -12 -13 -12 1 0
-32 -31 -32 -31 1 0
-75 -55 -75 -55 20 0
-19 -18 -19 -18 1 0
-31 -30 -31 -30 1 0
-30 -29 -30 -29 1 0
-27 -26 -27 -26 1 0
-22 -21 -22 -21 1 0
-3 -2 -3 -2 1 0
-12 -11 -12 -11 1 0
-9 -8 -9 -8 1 0
-26 -25 -26 -25 1 0
-1 0 -1 0 1 0
-29 -28 -29 -28 1 0
-7 -6 -7 -6 1 0
-10 -9 -10 -9 1 0
-4 -3 -4 -3 1 0
-17 -16 -17 -16 1 0
-15 -14 -15 -14 1 0
-102 -75 -102 -75 27 0
-35 -32 -35 -32 3 0
-23 -22 -23 -22 1 0
-18 -17 -18 -17 1 0
-28 -27 -28 -27 1 0
-20 -19 -20 -19 1 0
-5 -4 -5 -4 1 0
-21 -20 -21 -20 1 0
-16 -15 -16 -15 1 0
-2 -1 -2 -1 1 0
-6 -5 -6 -5 1 0
-25 -24 -25 -24 1 0
-14 -13 -14 -13 1 0
-55 -35 -55 -35 20 0
-24 -23 -24 -23 1 0
-11 -10 -11 -10 1 0


In [ ]:
# critical path 빨간색으로 채우고 시각화
for node in critical_path:
    main_network.nodes[node]['color'] = 'red'
    main_network.nodes[node]['style'] = 'filled'
    
src = graphviz.Source(nx.nx_pydot.to_pydot(main_network))
src.render("/content/gdrive/My Drive/Colab Notebooks/NetworkX/cpm_test2.gv", view=True)

'/content/gdrive/My Drive/Colab Notebooks/NetworkX/cpm_test2.gv.pdf'

In [ ]:
# 모든 공정의 여유시간 담은 리스트 생성
slack_list = []
for node in main_network.nodes:
    slack_list.append(main_network.nodes[node]['slack'])
# 최대 여유시간 반환
max_slack = max(slack_list)

In [ ]:
# 남은 여유 시간 정보를 시각화하기 위한 color list
color_list = ['red', 'orange', 'yellow', 'green', 'skyblue', 'blue', 'violet']

In [ ]:
# 최대 여유시간 대비 각 공정의 여유시간 비율을 색깔로 표현 후 시각화
for node in main_network.nodes:
    color = color_list[int(np.round(main_network.nodes[node]['slack'] / max_slack * 6))]
    main_network.nodes[node]['color'] = color
    
src = graphviz.Source(nx.nx_pydot.to_pydot(main_network))
src.render("/content/gdrive/My Drive/Colab Notebooks/NetworkX/cpm_test3.gv", view=True)

In [ ]:
# critical path 상의 activity 개수
len(critical_path)

36

In [ ]:
# critical path 상의 activity들의 총 소요 시간
critical_path_length

102

In [ ]:
# 모든 노드의 ES, EF, LS, LF, duration, Slack을 각각 출력
for node in main_network.nodes:
    print(main_network.nodes[node]['ES'], main_network.nodes[node]['EF'], main_network.nodes[node]['LS'], main_network.nodes[node]['LF'], main_network.nodes[node]['duration'], main_network.nodes[node]['slack'], main_network.nodes[node]['duration'])

-87 -78 -26 -17 9 61 9
-20 -19 -12 -11 1 8 1
-102 -80 -61 -39 22 41 22
-21 -20 -16 -15 1 5 1
-45 -41 -10 -6 4 35 4
-102 -83 -55 -36 19 47 19
-81 -70 -38 -27 11 43 11
-50 -49 -17 -16 1 33 1
-26 -25 -26 -25 1 0 1
-12 -11 -12 -11 1 0 1
-25 -24 -24 -23 1 1 1
-52 -49 -9 -6 3 43 3
-102 -79 -75 -52 23 27 23
-10 -9 -10 -9 1 0 1
-46 -43 -23 -20 3 23 3
-77 -67 -28 -18 10 49 10
-54 -53 -12 -11 1 42 1
-20 -19 -15 -14 1 5 1
-63 -47 -27 -11 16 36 16
-70 -67 -17 -14 3 53 3
-62 -45 -27 -10 17 35 17
-80 -65 -64 -49 15 16 15
-20 -19 -20 -19 1 0 1
-81 -70 -39 -28 11 42 11
-102 -87 -53 -38 15 49 15
-35 -34 -8 -7 1 27 1
-19 -18 -19 -18 1 0 1
-79 -78 -22 -21 1 57 1
-67 -54 -39 -26 13 28 13
-41 -40 -6 -5 1 35 1
-51 -47 -8 -4 4 43 4
-64 -56 -10 -2 8 54 8
-71 -70 -12 -11 1 59 1
-50 -47 -33 -30 3 17 3
-49 -48 -6 -5 1 43 1
-59 -54 -16 -11 5 43 5
-67 -64 -19 -16 3 48 3
-50 -48 -13 -11 2 37 2
-102 -80 -74 -52 22 28 22
-57 -56 -10 -9 1 47 1
-72 -60 -24 -12 12 48 12
-102 -87 -40 -25 15 62 15
-60 -57 -12 -9 3 48 3
-8